In [249]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm
import sqlite3
import time
import hashlib
import re
import datetime
import matplotlib.pyplot as plt
import matplotlib
import editdistance
import functools
from IPython.display import clear_output
import seaborn as sns
import random
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [250]:
from __future__ import division

import math
from bisect import bisect_left
from collections import namedtuple

In [744]:
fps = pd.read_csv("../tmp_data/inspector2014_with_dist2lib.csv")

In [745]:
fps.shape

(5827, 20)

In [747]:
fps.sample(1)

,idx,likely_libver,lib_ciphersuites,device_id,cipher_suites,extension_types,tls_version,device_vendor,device_name,cipher_code,ciphersuite_len,cipher_code_idxl,cipher_code_hex,updated_hexcodelis,updated_codelis,likely,likely_idx,likely_score,cumulative_agreement,distance_2lib_sim
638,50,curl_7.41.0_(x86_64-unknown-linux-gnu)_libcurl/7.41.0_CyaSSL/3.3.0,"[53, 47, 5, 4, 10]",s197bc0cc99,49171+49172+47+53+255,11+10+35+16+21,769,amazon,fire,"[49171, 49172, 47, 53, 255]",5,"['49171:0', '49172:1', '47:2', '53:3', '255:4']","['0xc013', '0xc014', '0x2f', '0x35', '0xff']","['0xc013', '0xc014', '0x2f', '0x35', '0xff']","[49171, 49172, 47, 53, 255]","[50, 0.25]",50,0.25,"[0.0, 0.0, 0.3333333333333333, 0.5, 0.4]",4


In [748]:
fps['extension_types'] = fps['extension_types'].fillna("missing")
fps['fps'] = fps.apply(lambda x: x['cipher_suites'] + "_" + x['extension_types'] + "_" + str(x['tls_version']), axis=1)

In [749]:
fps.loc[fps['device_vendor'] == 'sonos']['fps'].value_counts().shape

(30,)

### Add to subsection "TLS instance sharing and customizing"

In [704]:
fps_data = pd.read_csv("../tmp_data/fps_sample.csv")

In [707]:
fps_data.sample(3)

,fps,fps_hash,device_id,device_vendor,device_name,vendor_cnt,if_anon,if_export,if_rc4,if_rc2,if_des,if_nocipher,if_3des,tls_version,popularity,vul_score
291,769;49172+49162+57+56+55+54+136+135+134+133+49167+49157+53+132+49171+49161+51+50+49+48+154+153+152+151+69+68+67+66+49166+49156+47+150+65+7+49169+49159+49164+49154+5+4+49170+49160+22+19+16+13+49165+49155+10+255;11+10+35,944968447256267871,s0d2f015cd4,qnap,storage,2,0,0,54,0,0,0,81,769,Shared FPS,2
530,771;49196+49200+49195+49199+255;11+10+13+16,270463083062116128,s6e403abcd6,google,dot,2,0,0,0,0,0,0,0,771,Shared FPS,0
152,771;49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+154+153+152+151+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+150+65+49169+49159+49164+49154+5+49170+49160+22+19+16+13+49165+49155+10+255;11+10+13+15+13172+16+21,1831270424469231222,s0222d29760,roku,streamer,6,0,0,650,0,0,0,1170,771,Popular FPS,2


In [708]:
fps_pervendor = fps_data.groupby(['device_vendor'])['fps'].agg(lambda x: x.nunique()).reset_index(name='num_fps')

In [711]:
fps_pervendor.sort_values(by='num_fps', ascending=False)

,device_vendor,num_fps
0,amazon,244
15,google,172
48,synology,107
39,samsung,104
47,sony,97
...,...,...
54,tuya,1
13,epson,1
16,hdhomerun,1
42,silicondust,1


In [153]:
fps_pervendor = fps_fps.loc[fps_fps['num_vendor'] == 1].groupby(['vendor'])['fps'].agg(num_fps=lambda x: x.nunique(), \
                                                                                       set_fps=lambda x: set(x))\
                                                                                  .reset_index()

In [154]:
fps_pervendor.shape

(49, 3)

In [177]:
fps_data.sample(2)

,fps,fps_hash,device_id,device_vendor,device_name,vendor_cnt,if_anon,if_export,if_rc4,if_rc2,if_des,if_nocipher,if_3des,tls_version,popularity,vul_score
1366,771;53+49199+49171+49172+156+47+158+51+57+49169+10+5+4;65281+10+11+5+13+13172,-2788473907737668210,s9a85ea6ceb,tivo,mini,1,0,0,3,0,0,0,1,771,Exclusive FPS,2
658,771;52393+49195+49196+52392+49199+49200+49161+49162+49171+49172+156+157+47+53;23+65281+10+11+35+16+5+13,-959768159451643844,see7e2a4e2c,philips,led,2,0,0,0,0,0,0,0,771,Shared FPS,0


In [180]:
fps_data['ciphersuites'] = fps_data['fps'].apply(lambda x: x.split(";")[1])

In [182]:
fps_ciphersuite47 = fps_data.loc[fps_data['ciphersuites'] == "47"]

In [184]:
devid_47 = list(set(fps_ciphersuite47['device_id'].tolist()))

In [194]:
len(devid_47)

327

### Sharing and Customizing; SDK-related Analysis

In [1000]:
df = pd.read_csv("../tmp_data/ciphersuites_withpki.csv")

In [1001]:
df.shape

(27442, 33)

In [1002]:
df['device_id'].value_counts().shape

(2014,)

In [1005]:
import tldextract
blocked_domains = set()

def parse_blocklist():
    with open('hosts-blocklists.txt') as fp:
        for line in fp:
            if '::' in line:
                domain = line.split('/')[1].lower().strip()
                if domain:
                    blocked_domains.add(domain)

parse_blocklist()

In [1006]:
def is_hostname_blocked(sni):
    if sni in blocked_domains:
        return True
    else:
        return False

In [1007]:
print(is_hostname_blocked('doubleclick.com'))

True


In [1008]:
df['if_tracking'] = df['sni'].apply(lambda x: is_hostname_blocked(x))

In [1009]:
df['if_tracking'].value_counts()

False    27386
True        56
Name: if_tracking, dtype: int64

In [1010]:
df['extension_types_x'] = df['extension_types_x'].fillna("missing")

In [1011]:
df['fps'] = df.apply(lambda x: x['cipher_suites'] + "_" + x['extension_types_x'] + \
                             "_" + str(x['tls_version']), axis=1)

In [1012]:
df['fps'].value_counts().shape

(423,)

In [1013]:
df['device_vendor'].value_counts().shape

(66,)

In [1014]:
423/62

6.82258064516129

In [1019]:
df_filter = df.loc[df['device_vendor'] != 'Unknown']

In [1020]:
df_filter.sample(1)

,idx,likely_libver,lib_ciphersuites,device_id,cipher_suites,extension_types_x,tls_version,device_vendor,device_name,cipher_code,ciphersuite_len,cipher_code_idxl,cipher_code_hex,updated_hexcodelis,updated_codelis,likely,likely_idx,likely_score,cumulative_agreement,distance_2lib_sim,server_name_indication,response,server_tls_version,if_tls_fallback,cert_chain_len,chain_issuer_O,chain_issuer_CN,chain_subject_O,chain_subject_CN,cert_validity_days,version,extension_types_y,sni,if_tracking,fps
3606,48,curl_7.65.1_(x86_64-pc-linux-gnu)_libcurl/7.65.1_wolfSSL/3.7.1,"[49196, 49195, 49200, 49199, 159, 158, 52244, 52243, 52245, 49191, 49187, 49192, 49188, 49162, 49161, 49160, 49172, 49171, 49170, 107, 103, 57, 51]",s19277670e1,49195+49187+49196+49188+49199+49191+49200+49192+255,11+10+13+15+13172+16,771,roku,express,"[49195, 49187, 49196, 49188, 49199, 49191, 49200, 49192, 255]",9,"['49195:0', '49187:1', '49196:2', '49188:3', '49199:4', '49191:5', '49200:6', '49192:7', '255:8']","['0xc02b', '0xc023', '0xc02c', '0xc024', '0xc02f', '0xc027', '0xc030', '0xc028', '0xff']","['0xc02b', '0xc023', '0xc02c', '0xc024', '0xc02f', '0xc027', '0xc030', '0xc028', '0xff']","[49195, 49187, 49196, 49188, 49199, 49191, 49200, 49192, 255]","[48, 0.32]",48,0.32,"[0.0, 0.5, 0.6666666666666666, 0.5, 0.6, 0.5, 0.5714285714285714, 0.5, 0.4444444444444444, 0.5, 0.5714285714285714, 0.6363636363636364, 0.6956521739130435, 0.6666666666666666, 0.64, 0.6153846153846154, 0.5925925925925926, 0.5714285714285714, 0.5517241379310345, 0.5333333333333333, 0.5161290322580645, 0.5, 0.48484848484848486]",3,secure.netflix.com,False,TLS 1.2,False,2,['Netflix' 'VeriSign'],['Netflix Public SHA2 RSA CA 3'\n 'VeriSign Class 3 Public Primary Certification '],['Netflix' 'Netflix'],['secure.netflix.com' 'Netflix Public SHA2 RSA CA 3'],31 days 04:41:37,771,0+11+10+13+15+13172+16,secure.netflix.com,False,49195+49187+49196+49188+49199+49191+49200+49192+255_11+10+13+15+13172+16_771


In [1021]:
df_filter['sni'].value_counts().shape

(1151,)

In [1022]:
df_filter['fps'] = df_filter.apply(lambda x: x['cipher_suites'] + "_" + x['extension_types_x'] + \
                                              "_" + str(x['tls_version']), axis=1)

In [1023]:
df_sub = df_filter[['device_id', 'device_name' ,'device_vendor', 'cipher_suites', 'tls_version', 'cipher_code', \
                    'updated_codelis', 'server_name_indication', 'sni', 'fps', 'distance_2lib_sim', 'likely_libver']]

In [937]:
#df_corr = df_filter[['device_id', 'device_name' ,'device_vendor', 'cipher_suites', 'tls_version', 'cipher_code', \
#                    'updated_codelis', 'server_name_indication', 'sni', 'fps', 'response', 'server_tls_version', \
#                     'if_tls_fallback', 'cert_chain_len', 'chain_issuer_O', 'chain_issuer_CN', 'chain_subject_O', \
#                     'chain_subject_CN', 'cert_validity_days']]

In [1024]:
df_sub['updated_codelis'] = df_sub['updated_codelis'].apply(lambda x: eval(x))

<ipython-input-1024-f3465da35c76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['updated_codelis'] = df_sub['updated_codelis'].apply(lambda x: eval(x))


In [940]:
#df_corr['updated_codelis'] = df_corr['updated_codelis'].apply(lambda x: eval(x))

In [1025]:
vulmap = pd.read_csv('../tmp_data/vul_component_mapping.csv')

In [1026]:
vulmap.sample(1)

,ciphersuite,code,code_base10,cipher_y,key_exchange,mac,cipher,vul_components
87,0x22,"0x00,0x22",34,TLS_KRB5_WITH_DES_CBC_MD5,tls_krb5,md5,des_cbc,des


In [1027]:
vulmap['code_base10'] = vulmap['code_base10'].apply(lambda x: str(x))

In [1028]:
vulmap.loc[vulmap['code_base10'] == '15']['vul_components'].item()

'des'

In [1029]:
def find_vulcomp(code_lis):
    vul_lis = []
    for code in code_lis:
        try:
            vul_component = vulmap.loc[vulmap['code_base10'] == str(code)]['vul_components'].item()
            if vul_component not in vul_lis:
                vul_lis.append(vul_component)
        except:
            continue
    if vul_lis:
        return ",".join(vul_lis)
    else:
        return "none"

In [1030]:
df_sub['vul_comp'] = df_sub['updated_codelis'].apply(lambda x: find_vulcomp(x))

<ipython-input-1030-922977e6bb3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['vul_comp'] = df_sub['updated_codelis'].apply(lambda x: find_vulcomp(x))


In [847]:
#df_corr['vul_comp'] = df_corr['updated_codelis'].apply(lambda x: find_vulcomp(x))

<ipython-input-847-90e093a3a65a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['vul_comp'] = df_corr['updated_codelis'].apply(lambda x: find_vulcomp(x))


In [1031]:
df_sub.loc[df_sub['vul_comp'] != 'none'].shape

(18831, 13)

In [1032]:
#df_corr.loc[df_corr['vul_comp'] != 'none'].shape

In [1033]:
df_sub['domain'] = df_sub['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \
                                                 tldextract.extract(x).suffix)

<ipython-input-1033-4c20c3f149a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['domain'] = df_sub['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \


In [1034]:
lib_df = pd.read_csv('../tmp_data/library_fps.csv')

In [1035]:
lib_df.shape

(6891, 9)

In [1037]:
lib_df['fps'] = lib_df.apply(lambda x: x['cipher_suites'] + "_" + x['extension_types'] + \
                                              "_" + str(x['tls_version']), axis=1)

In [1038]:
lib_df = lib_df.drop_duplicates(subset=['fps'], keep='last').reset_index().drop(columns='index')

In [1039]:
lib_fpslis = lib_df['fps'].tolist()

In [1040]:
df_sub = df_sub.loc[~df_sub['fps'].isin(lib_fpslis)]

In [957]:
#df_corr.to_csv("../tmp_data/fps_sni.csv", index=False)

In [958]:
#df_sub.to_csv("../tmp_data/fps_sharing_customizing.csv", index=False)

In [1042]:
df_sub.sample(1)

,device_id,device_name,device_vendor,cipher_suites,tls_version,cipher_code,updated_codelis,server_name_indication,sni,fps,distance_2lib_sim,likely_libver,vul_comp,domain
17359,s95ccf22f96,bridge,wink,4867+4865+4866+52393+52392+49195+49199+49196+49200+49171+49172+156+157+47+53+10,771,"[4867, 4865, 4866, 52393, 52392, 49195, 49199, 49196, 49200, 49171, 49172, 156, 157, 47, 53, 10]","[4867, 4865, 4866, 52393, 52392, 49195, 49199, 49196, 49200, 49171, 49172, 156, 157, 47, 53, 10]",pi.ispot.tv,pi.ispot.tv,4867+4865+4866+52393+52392+49195+49199+49196+49200+49171+49172+156+157+47+53+10_23+65281+10+11+35+16+5+13+18+51+45+43+27+21_771,4,curl-7.60.076_openssl-1.1.1-pre2,3des,ispot.tv


In [1044]:
df_cnt0 = df_sub.groupby(['sni'])['fps'].agg(lambda x: x.nunique())\
                  .reset_index(name='num_fps')

In [1045]:
df_cnt1 = df_sub.groupby(['sni'])['device_id'].agg(lambda x: x.nunique())\
                  .reset_index(name='num_device')

In [1046]:
df_cnt2 = df_sub.groupby(['sni'])['device_vendor'].agg(lambda x: x.nunique())\
                   .reset_index(name='num_device_vendor')

In [1047]:
df_cnt3 = df_sub.groupby(['sni'])['device_vendor'].agg(lambda x: set(x))\
                   .reset_index(name='set_device_vendor')

In [1048]:
df_cnt4 = df_sub.groupby(['sni'])['vul_comp'].agg(lambda x: set(x))\
                .reset_index(name='set_vulcomp')

In [1049]:
df_cnt5 = df_sub.groupby(['sni'])['tls_version'].agg(lambda x: set(x))\
                .reset_index(name='set_tlsver')

In [1050]:
df_cnt_tmp = df_cnt0.merge(df_cnt1, how='inner', on='sni')
df_cnt_tmp = df_cnt_tmp.merge(df_cnt2, how='inner', on='sni')
df_cnt_tmp = df_cnt_tmp.merge(df_cnt3, how='inner', on='sni')
df_cnt_tmp = df_cnt_tmp.merge(df_cnt4, how='inner', on='sni')
df_cnt = df_cnt_tmp.merge(df_cnt5, how='inner', on='sni')

In [1051]:
df_cnt['sni'].value_counts().shape

(1131,)

In [1052]:
df_cnt.sample(2)

,sni,num_fps,num_device,num_device_vendor,set_device_vendor,set_vulcomp,set_tlsver
658,match.adsrvr.org,7,8,4,"{samsung, cisco, roku, amazon}","{rc,3des, none, 3des}",{771}
1105,www.gstatic.com,64,230,22,"{insignia, vizio, idevices, onkyo, wink, pioneer, lg, amcrest, google, cisco, philips, sharp, samsung, sony, amazon, hp, nvidia, lutron, xiaomi, tivo, ubiquity, roku}","{3des,rc, rc,3des, none, 3des}","{770, 771}"


In [1053]:
df_cnt.loc[(df_cnt['num_fps'] == 1)]['sni'].value_counts().shape

(207,)

In [1054]:
sdk = df_cnt.loc[(df_cnt['num_fps'] == 1) & (df_cnt['num_device_vendor'] > 1)]

In [1055]:
sdk['sni'].value_counts().shape

(56,)

In [1056]:
sdk_snilis = list(set(sdk['sni'].tolist()))

In [1057]:
sdk['domain'] = sdk['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \
                                                 tldextract.extract(x).suffix)

<ipython-input-1057-4cfc1f08abe0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdk['domain'] = sdk['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \


In [1058]:
sdk_domlis = list(set(sdk['domain'].tolist()))

In [1061]:
len(sdk_domlis)

27

In [1064]:
df_numdev = df_sub.groupby(['sni', 'fps', 'device_vendor'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1065]:
df_numdev['domain'] = df_numdev['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \
                                                 tldextract.extract(x).suffix)

In [1153]:
df_fpstmp0 = df_sub.loc[df_sub['sni'].isin(sdk_snilis)].groupby(['domain', 'fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_device')

In [1154]:
df_fpstmp1 = df_sub.loc[df_sub['sni'].isin(sdk_snilis)].groupby(['domain', 'fps'])['device_vendor'].agg(lambda x: set(x)).reset_index(name='set_vendor')

In [1156]:
df_fpstmp2 = df_sub.loc[df_sub['sni'].isin(sdk_snilis)].groupby(['domain', 'fps'])['vul_comp'].agg(lambda x: set(x)).reset_index(name='set_vulcomp')

In [1157]:
df_fpstmp = df_fpstmp0.merge(df_fpstmp1, how='inner', on=['domain', 'fps'])
df_fpstmp = df_fpstmp.merge(df_fpstmp2, how='inner', on=['domain', 'fps'])

In [1159]:
df_fpstmp[['domain', 'num_device', 'set_vendor', 'set_vulcomp']]

,domain,num_device,set_vendor,set_vulcomp
0,amazonmusic.com,3,"{sonos, google, amazon}",{none}
1,arlo.com,13,"{arlo, netgear}",{none}
2,cast4.audio,7,"{pioneer, onkyo}",{3des}
3,datadoghq.com,6,"{sense, texas instruments}",{none}
4,demdex.net,3,"{roku, tcl}","{rc,3des}"
5,dtvce.com,3,"{samsung, directv, humax}",{none}
6,googleapis.com,6,"{sony, nvidia}",{none}
7,googleusercontent.com,3,"{sony, nvidia}",{none}
8,hdhomerun.com,8,"{hdhomerun, silicondust}",{none}
9,hulu.com,3,"{roku, tcl}","{rc,3des}"


#### Vendors with num_dev == 1 visiting SNI with certain FP

In [1069]:
df_numdev.loc[(df_numdev['sni'].isin(sdk_snilis)) & \
              (df_numdev['num_dev'] == 1)].groupby(['domain'])['device_vendor'].agg(lambda x: set(x)).reset_index(name='set_vendor')

,domain,set_vendor
0,amazonmusic.com,"{sonos, google, amazon}"
1,datadoghq.com,{texas instruments}
2,demdex.net,{tcl}
3,dtvce.com,"{samsung, directv, humax}"
4,googleusercontent.com,{nvidia}
5,hdhomerun.com,{hdhomerun}
6,hulu.com,{tcl}
7,instagram.com,{cisco}
8,mgo-images.com,"{insignia, sharp}"
9,mgo.com,{insignia}


In [1071]:
df_sdk = df_sub.loc[df_sub['sni'].isin(sdk_snilis)][['sni', 'fps', 'domain']]

In [1072]:
df_sdk_dropdup = df_sdk.drop_duplicates(subset=['sni', 'fps'])

In [1073]:
df_sdk_dropdup.shape

(56, 3)

In [1139]:
df_merge = sdk.merge(df_sdk_dropdup, how='inner', on=['sni', 'domain'])

In [1141]:
df_merge[['domain', 'set_vulcomp', 'set_tlsver']].sort_values(by='domain')

,domain,set_vulcomp,set_tlsver
47,amazonmusic.com,{none},{771}
2,arlo.com,{none},{771}
36,arlo.com,{none},{771}
31,cast4.audio,{3des},{771}
1,datadoghq.com,{none},{771}
52,demdex.net,"{rc,3des}",{771}
9,dtvce.com,{none},{771}
40,googleapis.com,{none},{771}
38,googleusercontent.com,{none},{771}
33,hdhomerun.com,{none},{771}


In [1085]:
df_sdktemp1 = df_sdk_dropdup.groupby(['fps'])['domain'].agg(lambda x: set(x)).reset_index(name='samefps_domain_set')

In [1086]:
df_sdktemp2 = df_sdk_dropdup.groupby(['fps'])['sni'].agg(lambda x: x.nunique()).reset_index(name='samefps_num_sni')

In [1087]:
df_sdktemp_merge = df_sdktemp1.merge(df_sdktemp2, how='inner', on='fps')

In [1088]:
df_sdktemp_merge[['samefps_domain_set', 'samefps_num_sni']].sort_values(by='samefps_domain_set', ascending=True)

,samefps_domain_set,samefps_num_sni
0,{hdhomerun.com},2
15,"{sonos.com, pandora.com, amazonmusic.com}",7
14,{vizio.com},1
13,{roku.com},6
12,"{hulu.com, mgo.com, mgo-images.com, roku.com, ravm.tv, demdex.net}",15
11,{synology.com},1
10,"{arlo.com, netgear.com}",3
9,{skyegloup.com},2
8,{microsoft.com},2
7,"{googleusercontent.com, googleapis.com}",2


In [1094]:
df_domtmp = df_sub.loc[df_sub['sni'].isin(sdk_snilis)]

In [1160]:
df_domdev = df_domtmp.groupby(['domain', 'fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1162]:
df_domdev[['domain', 'num_dev']]

,domain,num_dev
0,amazonmusic.com,3
1,arlo.com,13
2,cast4.audio,7
3,datadoghq.com,6
4,demdex.net,3
5,dtvce.com,3
6,googleapis.com,6
7,googleusercontent.com,3
8,hdhomerun.com,8
9,hulu.com,3


In [1163]:
df_vendor1dev = df_sub.loc[(df_sub['sni'].isin(sdk_snilis))].groupby(['domain', 'fps', 'device_vendor'])['device_id']\
                      .agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1117]:
df_sub.loc[(df_sub['sni'].isin(sdk_snilis)) & \
           (df_sub['domain'] == "vizio.com") & \
           (df_sub['device_vendor'] == "wink")]['device_id'].value_counts()

s95ccf22f96    1
Name: device_id, dtype: int64

In [1167]:
df_vendor1dev.loc[(df_vendor1dev['num_dev'] == 1) & (df_vendor1dev['domain'] == 'roku.com')]

,domain,fps,device_vendor,num_dev
50,roku.com,49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+154+153+152+151+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+150+65+49169+49159+49164+49154+5+49170+49160+22+19+16+13+49165+49155+10+255_11+10+13+15+13172+16+21_771,cisco,1
55,roku.com,49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+65+49170+49160+22+19+16+13+49165+49155+10+255_11+10+35+13+15_771,brother,1
56,roku.com,49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+65+49170+49160+22+19+16+13+49165+49155+10+255_11+10+35+13+15_771,insignia,1
58,roku.com,49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+65+49170+49160+22+19+16+13+49165+49155+10+255_11+10+35+13+15_771,sharp,1


In [1174]:
df_sub.loc[(df_sub['domain'] == 'roku.com') & \
           (df_sub['fps'] == "49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+154+153+152+151+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+150+65+49169+49159+49164+49154+5+49170+49160+22+19+16+13+49165+49155+10+255_11+10+13+15+13172+16+21_771")]\
          .groupby(['device_vendor'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

,device_vendor,num_dev
0,brother,2
1,cisco,1
2,insignia,2
3,roku,99
4,sharp,4
5,tcl,17


In [1175]:
df_sub.loc[(df_sub['domain'] == 'roku.com') & \
           (df_sub['fps'] == "49200+49196+49192+49188+49172+49162+165+163+161+159+107+106+105+104+57+56+55+54+136+135+134+133+49202+49198+49194+49190+49167+49157+157+61+53+132+49199+49195+49191+49187+49171+49161+164+162+160+158+103+64+63+62+51+50+49+48+69+68+67+66+49201+49197+49193+49189+49166+49156+156+60+47+65+49170+49160+22+19+16+13+49165+49155+10+255_11+10+35+13+15_771")]\
          .groupby(['device_vendor'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

,device_vendor,num_dev
0,brother,2
1,cisco,1
2,insignia,2
3,roku,83
4,sharp,4
5,tcl,17


### If SNIs linked to FPs shared by multiple devices

In [1121]:
muldev = df_cnt.loc[(df_cnt['num_fps'] == 1) & (df_cnt['num_device'] > 1)]

In [1124]:
muldev_snilis = list(set(muldev['sni'].tolist()))

In [1125]:
muldev['domain'] = muldev['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \
                                                 tldextract.extract(x).suffix)

<ipython-input-1125-ffa0ee69e0c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muldev['domain'] = muldev['sni'].apply(lambda x: tldextract.extract(x).domain + "." + \


In [1126]:
df_muldev = df_sub.loc[df_sub['sni'].isin(muldev_snilis)][['sni', 'fps', 'domain']]

In [1127]:
df_muldev_dropdup = df_muldev.drop_duplicates(subset=['sni', 'fps'])

In [1128]:
df_muldev_dropdup.shape

(197, 3)

In [1129]:
df_merge = muldev.merge(df_muldev_dropdup, how='inner', on=['sni', 'domain'])

In [1136]:
df_merge.groupby(['domain'])['sni'].agg(lambda x: x.nunique()).reset_index(name='num_sni')[40:]

,domain,num_sni
40,nflximg.net,1
41,nflxso.net,1
42,nflxvideo.net,5
43,nintendo.com,1
44,nintendo.net,14
45,nist.gov,1
46,nvidiagrid.net,1
47,pandora.com,1
48,playstation.com,6
49,playstation.net,16


### Why devices have both FPS customization and sharing?

In [1240]:
fps.sample(1)

,idx,likely_libver,lib_ciphersuites,device_id,cipher_suites,extension_types,tls_version,device_vendor,device_name,cipher_code,ciphersuite_len,cipher_code_idxl,cipher_code_hex,updated_hexcodelis,updated_codelis,likely,likely_idx,likely_score,cumulative_agreement,distance_2lib_sim,fps
3360,39,curl-7.71.073_openssl-1.1.0l,"[49196, 49200, 159, 52393, 52392, 52394, 49195, 49199, 158, 49188, 49192, 107, 49187, 49191, 103, 49162, 49172, 57, 49161, 49171, 51, 157, 156, 61, 60, 53, 47, 255]",s8bc2818e69,49195+49196+49199+49200+49171+49172+156+157+47+53+255,11+10+35+13+16,771,amazon,echo,"[49195, 49196, 49199, 49200, 49171, 49172, 156, 157, 47, 53, 255]",11,"['49195:0', '49196:1', '49199:2', '49200:3', '49171:4', '49172:5', '156:6', '157:7', '47:8', '53:9', '255:10']","['0xc02b', '0xc02c', '0xc02f', '0xc030', '0xc013', '0xc014', '0x9c', '0x9d', '0x2f', '0x35', '0xff']","['0xc02b', '0xc02c', '0xc02f', '0xc030', '0xc013', '0xc014', '0x9c', '0x9d', '0x2f', '0x35', '0xff']","[49195, 49196, 49199, 49200, 49171, 49172, 156, 157, 47, 53, 255]","[39, 0.3793103448275862]",39,0.37931,"[0.0, 0.5, 0.3333333333333333, 0.5, 0.4, 0.3333333333333333, 0.42857142857142855, 0.5, 0.4444444444444444, 0.4, 0.36363636363636365, 0.3333333333333333, 0.32, 0.3076923076923077, 0.2962962962962963, 0.2857142857142857, 0.3448275862068966, 0.3333333333333333, 0.3225806451612903, 0.375, 0.36363636363636365, 0.4117647058823529, 0.45714285714285713, 0.4444444444444444, 0.43243243243243246, 0.47368421052631576, 0.5128205128205128, 0.55]",3,49195+49196+49199+49200+49171+49172+156+157+47+53+255_11+10+35+13+16_771


In [1241]:
fps = fps.loc[fps['device_vendor'] != 'unknown']

In [1242]:
fps['fps'].value_counts().shape

(903,)

In [1243]:
vendor_lis = list(set(fps['device_vendor'].tolist()))

In [1244]:
def divide_vendordev(df):
    df_ret = pd.DataFrame()
    for vendor in vendor_lis:
        df_temp = df.loc[df['device_vendor'] == vendor]
        total_devnum = len(list(set(df_temp['device_id'].tolist())))
        df_numdev = df_temp.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')
        df_setdev = df_temp.groupby(['fps'])['device_name'].agg(lambda x: set(x)).reset_index(name='set_dev')
        df_numdev = df_numdev.merge(df_setdev, how='inner', on='fps')
        df_numdev['device_vendor'] = vendor
        df_numdev['dev_num'] = total_devnum
        df_ret = pd.concat([df_ret, df_numdev])
    return df_ret

In [1245]:
df_vendordev = divide_vendordev(fps)

In [1246]:
df_vendordev.sample(1)

,fps,num_dev,set_dev,device_vendor,dev_num
85,52393+49195+49196+52392+49199+49200+49161+49162+49171+49172+156+157+47+53_65281+23+13+11+10_771,1,{bravia},sony,52


In [1247]:
df_vendordev.loc[(df_vendordev['device_vendor'] == 'amazon') & (df_vendordev['num_dev'] >= 10)]['set_dev']

148                                                      {echo, speaker, fire}
154                                               {stick, echo, speaker, fire}
160                                                       {echo, kindle, fire}
162                                              {stick, echo, streamer, fire}
164                        {echo, speaker, stick, dot, device, fire, streamer}
165                              {echo, dot, something, fire, video, streamer}
167                                               {stick, echo, speaker, fire}
168                                              {stick, echo, streamer, fire}
169    {echo, spot, stick, dot, switch, kindle, fire, device, voice, streamer}
171                                                      {echo, speaker, fire}
173                 {echo, speaker, stick, dot, device, fire, voice, streamer}
177       {echo, tickytackymain, stick, dot, something, fire, video, streamer}
183       {echo, tickytackymain, stick, dot, somethi

In [1248]:
df_vendordev.loc[(df_vendordev['device_vendor'] == 'amazon') & (df_vendordev['num_dev'] == 1)]['set_dev'].value_counts()

TypeError: unhashable type: 'set'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'set'


{echo}         88
{fire}         47
{ring}          9
{streamer}      5
{stick}         5
{dot}           4
{kindle}        4
{speaker}       3
{device}        1
{something}     1
{video}         1
Name: set_dev, dtype: int64

In [1269]:
amazon = fps.loc[fps['device_vendor'] == 'amazon'][['device_id', 'device_name', 'fps']]

In [1270]:
amazon['fps'].value_counts().shape

(244,)

In [1283]:
amazon = amazon.loc[(amazon['device_name'] != 'device') & (amazon['device_name'] != 'something') & (amazon['device_name'] != 'video')]

In [1284]:
amazon_gbtype = amazon.groupby(['fps'])['device_name'].agg(lambda x: x.nunique()).reset_index(name='num_devtype')

In [1285]:
amazon_gbtype.loc[amazon_gbtype['num_devtype'] > 1]['fps'].value_counts().shape

(64,)

In [1286]:
amazon_gbtype.loc[amazon_gbtype['num_devtype'] == 1]['fps'].value_counts().shape

(177,)

In [1287]:
echo = amazon.loc[amazon['device_name'] == 'echo']

In [1288]:
echo['fps'].value_counts().shape

(151,)

In [1289]:
echo.sample(1)

,device_id,device_name,fps
4751,scd47782fbc,echo,49195+49196+49199+49200+158+159+49161+49162+49171+49172+51+57+50+56+49159+49169+156+157+47+53+5+255_11+10+35+13_771


In [1262]:
echo_fpslis = list(set(echo['fps'].tolist()))

In [1263]:
amazon.loc[(amazon['fps'].isin(echo_fpslis)) & (amazon['device_name'] != 'echo')]['fps'].value_counts().shape

(60,)

In [1290]:
fps_all = pd.read_csv("../tmp_data/fps_sample.csv")

In [1291]:
fps_all.sample(1)

,fps,fps_hash,device_id,device_vendor,device_name,vendor_cnt,if_anon,if_export,if_rc4,if_rc2,if_des,if_nocipher,if_3des,tls_version,popularity,vul_score
490,771;49195+49199+49196+49200+52393+52392+52244+52243+49161+49171+49162+49172+156+157+47+53+10;65281+23+35+13+5+18+16+11+10,238357097510621,s7d18fc126b,panasonic,gzu,2,0,0,0,0,0,0,3,771,Shared FPS,1


In [1293]:
fps_sub = fps_all[['fps', 'if_anon', 'if_export', 'if_rc4', 'if_rc2', 'if_des', 'if_nocipher', 'if_3des', 'tls_version', 'vul_score']]

In [1297]:
fps_sub['fps'] = fps_all['fps'].apply(lambda x: "_".join([x.split(";")[1], x.split(";")[2], x.split(";")[0]]))

<ipython-input-1297-cd0e9fc8cb97>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fps_sub['fps'] = fps_all['fps'].apply(lambda x: "_".join([x.split(";")[1], x.split(";")[2], x.split(";")[0]]))


In [1298]:
echo_merge = echo.merge(fps_sub, on=['fps'], how='inner')

In [1299]:
echo_merge['fps'].value_counts().shape

(151,)

In [1300]:
echo_vul = echo_merge.loc[echo_merge['vul_score'] != 0]

In [1302]:
echo_vul['fps'].value_counts().shape

(40,)

In [1303]:
echovul_gbfps = echo_vul.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1304]:
echovul_gbfps.loc[echovul_gbfps['num_dev'] > 1]['fps'].value_counts().shape

(17,)

In [1304]:
echovul_gbfps.loc[echovul_gbfps['num_dev'] > 1]['fps'].value_counts().shape

(17,)

In [1305]:
echo_share = echo_merge.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1306]:
echo_share_lis = list(set(echo_share.loc[echo_share['num_dev'] > 1]['fps'].tolist()))

In [1307]:
echo_sharefps = echo_merge.loc[echo_merge['fps'].isin(echo_share_lis)]

In [1308]:
echo_sharefps['fps'].value_counts().shape

(41,)

In [1309]:
echo_sharefps.loc[echo_sharefps['vul_score'] != 0]['fps'].value_counts().shape

(17,)

In [1310]:
echo_single_lis = list(set(echo_share.loc[echo_share['num_dev'] == 1]['fps'].tolist()))

In [1311]:
echo_singlefps = echo_merge.loc[echo_merge['fps'].isin(echo_single_lis)]

In [1312]:
echo_singlefps['fps'].value_counts().shape

(110,)

In [1313]:
echo_singlefps.loc[echo_singlefps['vul_score'] != 0]['fps'].value_counts().shape

(23,)

In [1264]:
#amazon.to_csv("../tmp_data/amazon_device_type_conn.csv", index=False)

In [771]:
def get_graphdf(df):
    df_ret = pd.DataFrame()
    for vendor in vendor_lis:
        total_devnum = df.loc[df['device_vendor'] == vendor]['dev_num'].max()
        total_fpsnum = df.loc[df['device_vendor'] == vendor]['fps'].nunique()
        fps_1devnum = df.loc[(df['device_vendor'] == vendor) & (df['num_dev'] == 1)].shape[0]
        fps_10devnum = df.loc[(df['device_vendor'] == vendor) & (df['num_dev'] >= 10)].shape[0]
        df_tmp = pd.DataFrame([{'device_vendor':vendor, 'total_devnum':total_devnum, 'total_fpsnum':total_fpsnum, \
                                         'fps_1devnum':fps_1devnum, 'fps_10devnum':fps_10devnum}])
        df_ret = pd.concat([df_ret, df_tmp])
    return df_ret

In [772]:
df_graph = get_graphdf(df_vendordev)

In [773]:
df_graph.sort_values(by='total_fpsnum', ascending=False)[:10]

,device_vendor,total_devnum,total_fpsnum,fps_1devnum,fps_10devnum
0,amazon,521,244,168,30
0,google,467,172,112,19
0,synology,52,107,72,4
0,samsung,86,104,63,10
0,sony,52,97,56,6
0,lg,36,54,35,2
0,western digital,6,49,47,0
0,nvidia,22,43,20,4
0,tplink,29,39,34,1
0,roku,109,38,24,9


### Statistics to fillin

In [1176]:
fps_all = pd.read_csv("../tmp_data/fps_sample.csv")

In [1209]:
fps_all.sample(1)

,fps,fps_hash,device_id,device_vendor,device_name,vendor_cnt,if_anon,if_export,if_rc4,if_rc2,if_des,if_nocipher,if_3des,tls_version,popularity,vul_score
584,771;49200+49196+49192+49188+49172+49162+163+159+107+106+57+56+49177+49202+49198+49194+49190+49167+49157+157+61+53+49199+49195+49191+49187+49171+49161+162+158+103+64+51+50+49176+49201+49197+49193+49189+49166+49156+156+60+47+49170+49160+22+19+49175+49165+49155+10+255;11+10+13+15+21,-6153398580975328638,sa740c73226,vizio,speaker,4,21,0,0,0,0,0,56,771,Shared FPS,2


In [1314]:
fps_all['fps'].value_counts().shape

(901,)

In [1316]:
fps_all.loc[(fps_all['device_vendor'] == 'synology')]['fps'].value_counts().shape

(107,)

In [1315]:
fps_all.loc[(fps_all['device_vendor'] == 'synology') & \
            (fps_all['vul_score'] > 0)]['fps'].value_counts().shape

(81,)

In [1317]:
fps_all.loc[(fps_all['device_vendor'] == 'synology') & \
            (fps_all['vul_score'] > 0)]['device_id'].value_counts().shape

(19,)

In [1318]:
fps_all.loc[(fps_all['device_vendor'] == 'synology')]['device_id'].value_counts().shape

(22,)

In [1203]:
fps_singledev = fps_all.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1204]:
fps_singledev.loc[fps_singledev['num_dev'] == 1]['fps'].value_counts().shape

(698,)

In [1217]:
fps_singlevendor = fps_all.groupby(['fps'])['device_vendor'].agg(lambda x: x.nunique()).reset_index(name='num_vendor')

In [1218]:
fps_singlevendor_lis = list(set(fps_singlevendor.loc[fps_singlevendor['num_vendor'] == 1]['fps'].tolist()))

In [1195]:
fps_gbvendor = fps_all.groupby(['device_vendor'])['fps'].agg(lambda x: x.nunique()).reset_index(name='num_fps')

In [1198]:
fps_gbvendor['num_fps'].std()

41.243834037997075

In [1181]:
vul_fps = fps_all.loc[fps_all['vul_score'] != 0]

In [1200]:
vul_fps['fps'].value_counts().shape

(403,)

In [1182]:
vul_fps.groupby(['device_vendor'])['fps'].agg(lambda x: x.nunique()).reset_index(name='num_vulfps')['num_vulfps'].max()

81

In [1201]:
vul_fps_gbfps = vul_fps.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1202]:
vul_fps_gbfps.loc[vul_fps_gbfps['num_dev'] > 1]['fps'].value_counts().shape

(128,)

In [1189]:
fps_gbfps = fps_all.loc[fps_all['vul_score'] != 0].groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1190]:
fps_gbfps.loc[fps_gbfps['num_dev'] > 1]['fps'].value_counts().shape

(128,)

In [1191]:
fps_gbfps_lis = list(set(fps_gbfps.loc[fps_gbfps['num_dev'] > 1]['fps'].tolist()))

In [1193]:
fps_all.loc[fps_all['fps'].isin(fps_gbfps_lis)]['device_id'].value_counts().shape

(246,)

In [1194]:
fps_all.loc[fps_all['fps'].isin(fps_gbfps_lis)]['device_vendor'].value_counts().shape

(53,)

In [1185]:
broken_fps = fps_all.loc[(fps_all['if_anon'] != 0) | \
                         (fps_all['if_export'] != 0) | \
                         (fps_all['if_nocipher'] != 0)]

In [1211]:
broken_fps['fps'].value_counts().shape

(31,)

In [1186]:
broken_fps['device_id'].value_counts().shape

(27,)

In [1188]:
broken_fps['device_vendor'].value_counts()

synology           18
western digital     6
tplink              5
sony                4
amazon              3
hp                  2
lg                  2
samsung             2
qnap                1
vizio               1
philips             1
lutron              1
amcrest             1
google              1
Name: device_vendor, dtype: int64

In [1206]:
broken_gbfps = broken_fps.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1210]:
broken_gbfps.loc[broken_gbfps['num_dev'] > 1]['fps'].value_counts().shape

(9,)

In [1213]:
broken_fps['vul_score'].mean()

3.8958333333333335

In [1214]:
broken_fps['vul_score'].max()

7

In [1219]:
synology = fps_all.loc[fps_all['device_vendor'] == 'synology']

In [1220]:
synology.loc[(synology['fps'].isin(fps_singlevendor_lis)) & \
             (synology['vul_score'] > 1)]['fps'].value_counts().shape

(22,)

In [1276]:
echo = fps_all.loc[fps_all['device_name'] == 'echo']

In [1277]:
echo['fps'].value_counts().shape

(127,)

In [1278]:
echo_vul = echo.loc[echo['vul_score'] != 0]

In [1279]:
echo_vul.shape

(29, 16)

In [1281]:
echovul_gbfps = echo_vul.groupby(['fps'])['device_id'].agg(lambda x: x.nunique()).reset_index(name='num_dev')

In [1282]:
echovul_gbfps.loc[echovul_gbfps['num_dev'] > 1]['fps'].value_counts().shape

(0,)